In [ ]:
!pip install --uprgrade selenium

In [ ]:
import requests
from bs4 import BeautifulSoup
import csv

# Function to extract description from the startup link
def extract_description(startup_link):
    # Send a GET request to the startup link
    response = requests.get(startup_link)

    # Parse the HTML content
    soup = BeautifulSoup(response.text, 'html.parser')

    # Find the div containing the description
    description_div = soup.find('div', class_='startup__description')

    # Extract the description text
    description = description_div.text.strip() if description_div else ''

    return description

# Function to scrape startup cards from a given page URL
def scrape_startup_cards(url, csv_writer):
    # Send a GET request to the URL
    response = requests.get(url)

    # Parse the HTML content
    soup = BeautifulSoup(response.text, 'html.parser')

    # Find all the startup cards
    startup_cards = soup.find_all('div', class_='startupCard')

    # Check if there are no startup cards on this page
    if not startup_cards:
        return False

    # Loop through each startup card and extract information
    for card in startup_cards:
        # Extract the name of the startup
        startup_name_element = card.find('a', class_='block whitespace-nowrap text-ellipsis overflow-hidden font-medium')
        startup_name = startup_name_element.text

        # Generate the startup link
        startup_link = f"https://betalist.com{startup_name_element['href']}"

        # Extract the description
        description = extract_description(startup_link)

        # Write the data to the CSV file
        csv_writer.writerow({'Name': startup_name, 'Description': description})

    return True

# Base URL of the webpage to scrape
base_url = 'https://betalist.com/topics/startups'
page_number = 1

# Create a CSV file to save the data
csv_file_path = 'startup_data.csv'
with open(csv_file_path, 'w', newline='', encoding='utf-8') as csv_file:
    fieldnames = ['Name', 'Description']
    csv_writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
    csv_writer.writeheader()

    # Scrape startup cards from all pages
    while True:
        page_url = f"{base_url}?page={page_number}"
        print(f"Scraping page {page_number}...")
        if not scrape_startup_cards(page_url, csv_writer):
            print("No more startup cards found. Stopping the scraping process.")
            break

        page_number += 1

print(f"Data has been successfully saved to '{csv_file_path}'.")


Scraping page 1...
Scraping page 2...
Scraping page 3...
Scraping page 4...
Scraping page 5...


In [ ]:
import pandas as pd

# Load the main dataset CSV file
main_dataset_path = "/content/g2_finaldata.csv"  # Update with the path to your main dataset CSV file
main_df = pd.read_csv(main_dataset_path)

# Load the dataset with fields "name" and "description"
provided_dataset_path = "/content/startup_data.csv"  # Update with the path to your provided dataset CSV file
provided_df = pd.read_csv(provided_dataset_path)

# Merge the two datasets based on name and description
merged_df = pd.merge(main_df, provided_df, on=["name", "description"], how="left", indicator=True)

# Filter out the products that are only in the main dataset
missing_products = merged_df.loc[merged_df["_merge"] == "left_only", main_df.columns]

# Display the list of missing products
print("Missing Products:")
print(missing_products)


Missing Products:
      product_type                            name  \
0         Software  Simple Custom Reports for Jira   
1         Software  Ampliz Healthcare Intelligence   
2         Provider                      MAKRAM SRL   
3         Software                          Isogen   
4         Software                     GetJobsDone   
...            ...                             ...   
43335     Software                      Klient PSA   
43336     Software                          Midesk   
43337     Software                          WRKTOP   
43338     Software                           Later   
43339     Software                   Apideck Unify   

                                             description  
0      A powerful tool that allows you to create cust...  
1      Ampliz, the leading healthcare data provider, ...  
2                                                    NaN  
3      Isogen is the world's leading solution for the...  
4      GetJobsDone is a platform with 

In [ ]:
import pandas as pd

# Load the main dataset CSV file
main_dataset_path = "/content/g2_finaldata.csv"  # Update with the path to your main dataset CSV file
main_df = pd.read_csv(main_dataset_path)

# Load the dataset with fields "name" and "description"
provided_dataset_path = "/content/startup_data.csv"  # Update with the path to your provided dataset CSV file
provided_df = pd.read_csv(provided_dataset_path)

# Extract the unique products from both datasets
main_products = set(zip(main_df["name"], main_df["description"]))
provided_products = set(zip(provided_df["name"], provided_df["description"]))

# Find the products in the provided dataset that are not in the main dataset
missing_products = provided_products - main_products

# Convert the missing products set to a DataFrame
missing_products_df = pd.DataFrame(list(missing_products), columns=["name", "description"])

# Save the list of missing products to a CSV file
missing_products_csv_path = "missing_products.csv"
missing_products_df.to_csv(missing_products_csv_path, index=False)

# Display the list of missing products
print("Missing Products:")
print(missing_products_df)

print(f"The list of missing products has been saved to '{missing_products_csv_path}'.")



Missing Products:
                         name  \
0     Ok2 Smart Shortener URL   
1             Marketing Stack   
2                   SESMetric   
3                    Inoffice   
4              Get Tech Press   
...                       ...   
3628        Quaderno Checkout   
3629                   Opesta   
3630                     Qoop   
3631                 ROCKY.AI   
3632                   Modify   

                                            description  
0     Ok2 Smart Shortener URL is Build, Customize & ...  
1     Marketing Stack is the easiest way to filter t...  
2     SES Metricg was started with main mission to p...  
3     Inoffice is a collaboration tool for remote de...  
4     Get Tech Press solves the pain of getting pres...  
...                                                 ...  
3628  Quaderno Checkout is a simple widget to provid...  
3629  Opesta allows users to create detailed marketi...  
3630  Qoop is a gamified social Q&A where users earn...  
3631 

In [ ]:
import pandas as pd

# Load the main dataset CSV file
main_dataset_path = "/content/g2_finaldata.csv"  # Update with the path to your main dataset CSV file
main_df = pd.read_csv(main_dataset_path)

# Load the dataset with fields "name" and "description"
provided_dataset_path = "/content/startup_data.csv"  # Update with the path to your provided dataset CSV file
provided_df = pd.read_csv(provided_dataset_path)

# Extract the unique products from both datasets
main_products = set(zip(main_df["name"], main_df["description"]))
provided_products = set(zip(provided_df["name"], provided_df["description"]))

# Find the products in the provided dataset that are not in the main dataset
missing_products = provided_products - main_products

# Find the common products between both datasets
common_products = provided_products.intersection(main_products)

# Convert the missing products set to a DataFrame
missing_products_df = pd.DataFrame(list(missing_products), columns=["name", "description"])

# Convert the common products set to a DataFrame
common_products_df = pd.DataFrame(list(common_products), columns=["name", "description"])

# Save the list of missing products to a CSV file
missing_products_csv_path = "missing_productstry.csv"
missing_products_df.to_csv(missing_products_csv_path, index=False)

# Save the list of common products to a CSV file
common_products_csv_path = "common_productstry.csv"
common_products_df.to_csv(common_products_csv_path, index=False)

# Display the list of missing products
print("Missing Products:")
print(missing_products_df)

print(f"The list of missing products has been saved to '{missing_products_csv_path}'.")

# Display the list of common products
print("\nCommon Products:")
print(common_products_df)

print(f"The list of common products has been saved to '{common_products_csv_path}'.")


Missing Products:
                         name  \
0     Ok2 Smart Shortener URL   
1             Marketing Stack   
2                   SESMetric   
3                    Inoffice   
4              Get Tech Press   
...                       ...   
3628        Quaderno Checkout   
3629                   Opesta   
3630                     Qoop   
3631                 ROCKY.AI   
3632                   Modify   

                                            description  
0     Ok2 Smart Shortener URL is Build, Customize & ...  
1     Marketing Stack is the easiest way to filter t...  
2     SES Metricg was started with main mission to p...  
3     Inoffice is a collaboration tool for remote de...  
4     Get Tech Press solves the pain of getting pres...  
...                                                 ...  
3628  Quaderno Checkout is a simple widget to provid...  
3629  Opesta allows users to create detailed marketi...  
3630  Qoop is a gamified social Q&A where users earn...  
3631 